In [1]:
import torch
import src
import wandb
import numpy as np
import os
import cv2

from PIL import Image
from sklearn.model_selection import KFold
from src.utils.dataset import Preprocessing

In [2]:
ENCODER = 'resnet152'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu') 
PARAMS = {
    "lr" : 0.0005,
    "batch_size" : 1,
    "num_splits" : 4,
}

In [3]:
loss = src.utils.base.SumOfLosses(
    src.utils.losses.DiceLoss(),
    src.utils.losses.BCELoss()
)

metrics = [
    src.utils.metrics.IoU(threshold=0.5)
]

In [4]:
wandb.login()
wandb.init(project="Trus_images")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dalyutkin01. Use `wandb login --relogin` to force relogin


In [5]:
preprocess = Preprocessing(
    dir='./data/',
    img_size=768,
    augmentation=False
)

In [6]:
clear_images, clear_masks = preprocess.load_folder()
kfold = KFold(n_splits=PARAMS["num_splits"], shuffle=True)
fold_data = []
for fold, (train_indices, test_indices) in enumerate(kfold.split(clear_masks)):
    train_data = {
        "images" : [clear_images[index] for index in train_indices],
        "masks" : [clear_masks[index] for index in train_indices]
    }
    test_data = {
        "images" : [clear_images[index] for index in test_indices],
        "masks" : [clear_masks[index] for index in test_indices]
    }
    fold_data.append((train_data, test_data))

In [7]:
track = 0
TEST_IMAGES = 0
TEST_MASKS = 0
loader_for_top_model = 0
for fold, (train_data, test_data) in enumerate(fold_data):
    print(f"Training on fold {fold + 1}...")
    model = src.FPN(
        encoder_name=ENCODER, 
        encoder_weights=ENCODER_WEIGHTS,
        in_channels=1,
        classes=1,
        activation=ACTIVATION,
    )
    optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=PARAMS["lr"]),])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
        factor=0.1, patience=10, threshold=0.0001, threshold_mode='abs')
    train_epoch = src.utils.train.TrainEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        optimizer=optimizer,
        device=DEVICE,
        verbose=True,
    )
    valid_epoch = src.utils.train.ValidEpoch(
        model,
        loss=loss, 
        metrics=metrics, 
        device=DEVICE,
        verbose=True,
    )
    
    train_dataloader, valid_dataloader = preprocess.Generator(
        batch_size=PARAMS["batch_size"],
        X_train=train_data["images"],
        y_train=train_data["masks"],
        X_test=test_data["images"],
        y_test=test_data["masks"]
    )
    for epoch in range(0, 15):
        print('\nEpoch: {}'.format(epoch + 1))
        train_logs = train_epoch.run(train_dataloader)
        valid_logs = valid_epoch.run(valid_dataloader)
        
        wandb.log({
            f'fold_{fold+1}/train/train_IoU': train_logs['iou_score'],
            f'fold_{fold+1}/train/train_loss': train_logs['dice_loss + bce_loss']
        })
        wandb.log({
            f'fold_{fold+1}/valid/valid_IoU': valid_logs['iou_score'], 
            f'fold_{fold+1}/valid/valid_loss': valid_logs['dice_loss + bce_loss']
        })
        
        scheduler.step(valid_logs['dice_loss + bce_loss'])
        if track < valid_logs['iou_score']:
            TEST_IMAGES = test_data["images"]
            TEST_MASKS = test_data["masks"]
            track = valid_logs['iou_score']
            loader_for_top_model = test_data
            torch.save({
                'model_state_dict': model.state_dict(),
                'best_IoU' : track
                }, './checkpoint/best_model_kfold.pth')
            print('Model saved!')
wandb.finish()

Training on fold 1...

Epoch: 1
valid: 100%|██████████| 26/26 [00:10<00:00,  2.50it/s, dice_loss + bce_loss - 0.8223, iou_score - 0.1994]
Model saved!

Epoch: 2
valid: 100%|██████████| 26/26 [00:10<00:00,  2.50it/s, dice_loss + bce_loss - 0.6743, iou_score - 0.2788]
Model saved!

Epoch: 3
valid: 100%|██████████| 26/26 [00:10<00:00,  2.45it/s, dice_loss + bce_loss - 0.708, iou_score - 0.2661] 

Epoch: 4
valid: 100%|██████████| 26/26 [00:10<00:00,  2.50it/s, dice_loss + bce_loss - 0.5974, iou_score - 0.347] 
Model saved!

Epoch: 5
valid: 100%|██████████| 26/26 [00:10<00:00,  2.51it/s, dice_loss + bce_loss - 0.6389, iou_score - 0.31]  

Epoch: 6
valid: 100%|██████████| 26/26 [00:10<00:00,  2.52it/s, dice_loss + bce_loss - 0.6424, iou_score - 0.3114]

Epoch: 7
valid: 100%|██████████| 26/26 [00:10<00:00,  2.54it/s, dice_loss + bce_loss - 0.6708, iou_score - 0.2772]

Epoch: 8
valid: 100%|██████████| 26/26 [00:10<00:00,  2.56it/s, dice_loss + bce_loss - 0.6771, iou_score - 0.2894]

Epoch: 9
v

fold_1/train/train_IoU,▁▂▃▃▄▅▅▆▆▆▇▇▇▇█
fold_1/train/train_loss,█▆▆▅▅▄▃▃▂▂▂▂▂▁▁
fold_1/valid/valid_IoU,▂▅▅█▇▇▅▆▄▅▄▂▄▅▁
fold_1/valid/valid_loss,▇▃▄▁▂▂▃▃▆▄▅▆▆▄█
fold_2/train/train_IoU,▁▃▂▃▄▅▅▆▆▆▆▆▇▇█
fold_2/train/train_loss,█▆▆▅▄▄▄▃▃▂▂▂▂▂▁
fold_2/valid/valid_IoU,▂█▆▇▃▆▃▂▇▁▂▅▄▅▅
fold_2/valid/valid_loss,█▁▂▂▅▄▅▆▂█▇▄▅▄▄
fold_3/train/train_IoU,▁▂▃▃▄▄▅▅▆▆▇▇▇▇█
fold_3/train/train_loss,█▇▆▆▅▅▃▃▃▂▂▂▂▂▁
fold_3/valid/valid_IoU,▇▆█▇▅▅▅▆▁▄▂▅▆▃▂


In [14]:
checkpoint = torch.load('./checkpoint/best_model_kfold.pth')
# 

In [9]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [10]:
shape = TEST_IMAGES[0].shape[:2]
preds = []
for image in TEST_IMAGES:
    image_prepoc = preprocess.prepocessing_image(image)
    image_prepoc = torch.tensor(np.array([[image_prepoc]]), dtype = torch.float)
    image_prepoc = image_prepoc.to(DEVICE)
    with torch.no_grad():
        predict = model.forward(image_prepoc)
    predict_value = predict.cpu().detach().numpy()
    preds.append(predict_value)

In [11]:
valid_pred_folder = './valid_predict_kfold_v1/'

In [12]:
def preproc_predict(image):
    predict = (image[0][0]*255).astype(np.uint8)
    img  = np.array(Image.fromarray(predict))
    return img

def green_chanel(image):
    blue,green,red = cv2.split(image)
    zeros = np.zeros(blue.shape, np.uint8)
    greenBGR = cv2.merge([zeros,green,zeros])
    return greenBGR

def blue_chanel_predict(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (1280, 872), interpolation = cv2.INTER_AREA)
    blue, _, _ = cv2.split(image)
    zeros = np.zeros(blue.shape, np.uint8)
    blueBGR = cv2.merge([blue,zeros,zeros])
    return blueBGR

def green_chanel_predict(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (1280, 872), interpolation = cv2.INTER_AREA)
    _, green, _ = cv2.split(image)
    zeros = np.zeros(green.shape, np.uint8)
    green = cv2.merge([zeros,green,zeros])
    return green

def calc_IoU(image_first, image_second):
    bitwiseOR = cv2.bitwise_or(image_first, image_second)
    bitwiseOR = bitwiseOR / 255
    bitwiseOR = (np.rint(bitwiseOR)).astype(int)
    summary = bitwiseOR.sum()
    bitwiseAnd = cv2.bitwise_and(image_first, image_second)
    bitwiseAnd = bitwiseAnd / 255
    bitwiseAnd = (np.rint(bitwiseAnd)).astype(int)
    peres = bitwiseAnd.sum()
    return peres / summary

In [13]:
for num in range(len(TEST_IMAGES)):
    true_mask = TEST_MASKS[num]
    greenBGR = green_chanel(true_mask)
    concat_true = cv2.addWeighted(TEST_IMAGES[num], 0.4, TEST_MASKS[num], 0.9, 0)
    
    #resize true image and create 3rd channels
    cvtColor = cv2.cvtColor(TEST_IMAGES[num], cv2.COLOR_BGR2GRAY )
    resize_image = cv2.resize(cvtColor, (1280, 872), interpolation = cv2.INTER_AREA)
    resize_image = cv2.cvtColor(resize_image, cv2.COLOR_BGR2RGB)
    
    #preproc predict
    predict_mask = preproc_predict(preds[num])
    blueBGR_predict = blue_chanel_predict(predict_mask)
    greenBGR_predict = green_chanel_predict(predict_mask)
    # concat_predict = cv2.bitwise_or(resize_image, predict_mask, mask = predict_mask)
    # image_rgb=cv2.cvtColor(concat_predict, cv2.COLOR_BGR2RGB)
    
    #calc IoU
    # print(greenBGR.shape)
    # print(greenBGR_predict.shape)
    IoU = calc_IoU(greenBGR, greenBGR_predict)
    
    #concat two masks
    bitwiseAnd = cv2.bitwise_or(greenBGR, blueBGR_predict)
    
    #concat true images with mask
    concat_predict = cv2.addWeighted(resize_image, 0.4, bitwiseAnd, 0.7, 0)
    concat_predict_resize = cv2.resize(concat_predict, (1280, 872), interpolation = cv2.INTER_AREA)
    
    #put text
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (570, 50)
    fontScale = 1
    color = (0, 255, 255)
    thickness = 2
    image = cv2.putText(concat_predict_resize, 'IoU ' + str(round(IoU, 3)), org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)
    
    #save 
    cv2.imwrite(filename=os.path.join(valid_pred_folder + str(num) + "_true" + '.png'), img=concat_true)
    cv2.imwrite(filename=os.path.join(valid_pred_folder + str(num) + "_predict" + '.png'), img=concat_predict_resize)